In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import json
import random

In [ ]:
with open('data/california_counties.json') as f:
  cali_geojson = json.load(f)

df = pd.DataFrame([
  [
    feature['properties']['NAME'], 
    random.random()
  ] for feature in cali_geojson['features']
], columns=['NAME', 'FIRE_PROBABILITY'])

car_df = px.data.carshare()

locations = json.load(open('data/locations.json'))
locations_df = pd.DataFrame(
  [location + [random.random()] for location in locations],
  columns=['LAT', 'LON', 'FIRE_SCORE']
)

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = ff.create_hexbin_mapbox(
    data_frame=locations_df, lat="LAT", lon="LON",
    nx_hexagon=10, opacity=0.5, labels={"color": "FIRE_SCORE"},
    color="FIRE_SCORE", agg_func=np.mean,
    min_count=1
)

fig.show()

In [ ]:
fig = px.choropleth(df, geojson=cali_geojson,
                    locations="NAME", featureidkey="properties.NAME",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig.layout.geo